In [6]:
pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.5 MB/s eta 0:00:00


**The assembly API key is initialized.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json

api_key = ''
file_path = '/content/API.json'

try:
    with open(file_path, 'r') as file_to_read:
        json_data = json.load(file_to_read)
        api_key = json_data["API_KEY"]

except FileNotFoundError:
    print("JSON file not found at the specified path:", file_path)

except KeyError:
    print("API_KEY key not found in the JSON data.")

except json.JSONDecodeError as e:
    print("Error decoding JSON data:", str(e))

**Install python library for YouTube**

**Importing YouTube library from pytube. The user gets 2 options such that it can upload a youtube video or else it can upload video from the drive to get audio from the video.**

In [9]:
print('For Youtube video enter 1 and for drive video enter 0')
n=int(input('ENTER = '))
if n==1:
  link=input('ENTER THE LINK : ')
  from pytube import YouTube
  video=YouTube(link)
  yt=video.streams.get_audio_only()
  k=yt.download()
elif n==0:
  link=input('ENTER THE LINK : ')
  k=link
else:
  print('Sorry wrong input.')

For Youtube video enter 1 and for drive video enter 0
ENTER = 0
ENTER THE LINK : /content/drive/MyDrive/Practice English Conversation  Need a New Show.3gpp


**Uploading the audio data retrieved from the YouTube video and feeding into the model to get the transcript.**

In [10]:
import sys
import time
import requests

file1 = k
def read_file(file1, size = 5242880):
  with open(file1, 'rb') as file:
    while True:
      data = file.read(size)
      if not data:
        break
      yield data
headers = {'authorization' : api_key}
response = requests.post('https://api.assemblyai.com/v2/upload', headers = headers, data = read_file(file1))
audio_url = response.json()['upload_url']
print(audio_url)

https://cdn.assemblyai.com/upload/0e509f22-6028-42c8-a41b-b4770433d4ac


**This step performs authorization and perform transcript input response**

In [11]:
endpoint = "https://api.assemblyai.com/v2/transcript"

json = {"audio_url": audio_url}

headers = {"authorization" : api_key, "content_type": "application/json"}

transcript_input_response = requests.post(endpoint, json = json, headers = headers)

temp1 = transcript_input_response.json()
temp1

{'id': '6xzmu3c7dn-ed26-4da5-800a-64069c16284b',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'queued',
 'audio_url': 'https://cdn.assemblyai.com/upload/0e509f22-6028-42c8-a41b-b4770433d4ac',
 'text': None,
 'words': None,
 'utterances': None,
 'confidence': None,
 'audio_duration': None,
 'punctuate': True,
 'format_text': True,
 'dual_channel': None,
 'webhook_url': None,
 'webhook_status_code': None,
 'webhook_auth': False,
 'webhook_auth_header_name': None,
 'speed_boost': False,
 'auto_highlights_result': None,
 'auto_highlights': False,
 'audio_start_from': None,
 'audio_end_at': None,
 'word_boost': [],
 'boost_param': None,
 'filter_profanity': False,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redact_pii_policies': None,
 'redact_pii_sub': None,
 'speaker_labels': False,
 'content_safety': False,
 'iab_categories': False,
 'content_safety_labels': {},
 'iab_cat

In [12]:
transcript_id = transcript_input_response.json()["id"]
transcript_id

'6xzmu3c7dn-ed26-4da5-800a-64069c16284b'

**The transcript output status is reflected. The status reflected can be processing or completed.**

In [29]:
endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
headers = {"authorization": api_key, }

condition='not started'
while condition!='completed':
    transcript_output_response = requests.get(endpoint, headers = headers)
    condition = transcript_output_response.json()["status"]

In [30]:
temp2 = transcript_output_response.json()
temp2

{'id': '6xzmu3c7dn-ed26-4da5-800a-64069c16284b',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'completed',
 'audio_url': 'https://cdn.assemblyai.com/upload/0e509f22-6028-42c8-a41b-b4770433d4ac',
 'text': "Let's practice. English conversation. What are we feeling tonight? Honestly? We need a new show. I'm feeling burnt out on the current series. Burnt out means you're not interested in it anymore. You're tired of it. AW, really? Yeah, I'm burnt out. I'm just bored with it. Why? I don't know. Too much violence. I'm just over it. When you're over something, you don't want to have anything to do with it anymore. You're tired of it. It doesn't interest you. Well, I hope you're not over hanging out with me. Never.",
 'words': [{'text': "Let's",
   'start': 410,
   'end': 634,
   'confidence': 0.94702,
   'speaker': None},
  {'text': 'practice.',
   'start': 682,
   'end': 1258,
   'confidence': 0.99985,
   'speaker': 

In [31]:
import datetime
for segment in temp2['words']:
    start_time = segment['start']
    end_time = segment['end']

    time_format1 = datetime.timedelta(milliseconds=start_time)
    time_string1 = str(time_format1)
    time_string1 = time_string1[:-7] + ',' + time_string1[-6:]
    segment['start'] = time_string1.zfill(15)[:-3]

    time_format2 = datetime.timedelta(milliseconds=end_time)
    time_string2 = str(time_format2)
    time_string2 = time_string2[:-7] + ',' + time_string2[-6:]
    segment['end'] = time_string2.zfill(15)[:-3]
temp2['words']

[{'text': "Let's",
  'start': '00:00:00,410',
  'end': '00:00:00,634',
  'confidence': 0.94702,
  'speaker': None},
 {'text': 'practice.',
  'start': '00:00:00,682',
  'end': '00:00:01,258',
  'confidence': 0.99985,
  'speaker': None},
 {'text': 'English',
  'start': '00:00:01,354',
  'end': '00:00:01,882',
  'confidence': 0.99918,
  'speaker': None},
 {'text': 'conversation.',
  'start': '00:00:01,946',
  'end': '00:00:02,842',
  'confidence': 0.99972,
  'speaker': None},
 {'text': 'What',
  'start': '00:00:02,986',
  'end': '00:00:03,246',
  'confidence': 0.99999,
  'speaker': None},
 {'text': 'are',
  'start': '00:00:03,268',
  'end': '00:00:03,406',
  'confidence': 0.99991,
  'speaker': None},
 {'text': 'we',
  'start': '00:00:03,428',
  'end': '00:00:03,614',
  'confidence': 0.99999,
  'speaker': None},
 {'text': 'feeling',
  'start': '00:00:03,652',
  'end': '00:00:03,994',
  'confidence': 0.99986,
  'speaker': None},
 {'text': 'tonight?',
  'start': '00:00:04,042',
  'end': '00:

In [37]:
transcript_with_timeframes = temp2['words']
output_data = []  # List to store the output sentences

sentence = ''
flag = True

for segment in transcript_with_timeframes:
    start_time = segment['start']
    end_time = segment['end']
    text = segment['text']

    if flag:
        output_data.append(start_time+" --> ")

    if text[-1] != '.' and text[-1] != '?':
        sentence += text + " "
        flag = False

    else:
        sentence += text
        output_data.append(end_time+"\n"+sentence+"\n")  # Store the sentence in the output_data list
        #output_data.append(end_time)
        sentence = ''
        flag = True

#Convert the list into a single string with line breaks

data = ''.join(output_data)
print(data)


00:00:00,410 --> 00:00:01,258
Let's practice.
00:00:01,354 --> 00:00:02,842
English conversation.
00:00:02,986 --> 00:00:04,490
What are we feeling tonight?
00:00:04,570 --> 00:00:05,130
Honestly?
00:00:05,210 --> 00:00:06,350
We need a new show.
00:00:06,420 --> 00:00:09,386
I'm feeling burnt out on the current series.
00:00:09,578 --> 00:00:12,714
Burnt out means you're not interested in it anymore.
00:00:12,762 --> 00:00:13,998
You're tired of it.
00:00:14,084 --> 00:00:15,280
AW, really?
00:00:15,730 --> 00:00:17,166
Yeah, I'm burnt out.
00:00:17,188 --> 00:00:19,070
I'm just bored with it.
00:00:19,220 --> 00:00:19,920
Why?
00:00:20,530 --> 00:00:21,614
I don't know.
00:00:21,732 --> 00:00:22,938
Too much violence.
00:00:23,034 --> 00:00:24,774
I'm just over it.
00:00:24,932 --> 00:00:29,218
When you're over something, you don't want to have anything to do with it anymore.
00:00:29,314 --> 00:00:30,374
You're tired of it.
00:00:30,412 --> 00:00:32,230
It doesn't interest you.
00:0

**Gives the output of the transcript.**

In [39]:
data1 = transcript_output_response.json()["text"]
data1

"Let's practice. English conversation. What are we feeling tonight? Honestly? We need a new show. I'm feeling burnt out on the current series. Burnt out means you're not interested in it anymore. You're tired of it. AW, really? Yeah, I'm burnt out. I'm just bored with it. Why? I don't know. Too much violence. I'm just over it. When you're over something, you don't want to have anything to do with it anymore. You're tired of it. It doesn't interest you. Well, I hope you're not over hanging out with me. Never."

**Enables user to download the transcript for the corresponding video.**

In [40]:
language_codes = {
    'english': 'en',
    'spanish': 'es',
    'chinese': 'zh',
    'hindi': 'hi',
    'arabic': 'ar',
    'portuguese': 'pt',
    'bengali': 'bn',
    'russian': 'ru',
    'japanese': 'ja',
    'punjabi': 'pa',
    'german': 'de',
    'french': 'fr',
    'italian': 'it',
    'korean': 'ko',
    'turkish': 'tr',
    'telugu': 'te',
    'marathi': 'mr',
    'tamil': 'ta',
    'vietnamese': 'vi',
    'urdu': 'ur'
}

In [41]:
from googletrans import Translator

def translate_english_to_other(text,ln):
    if text is None or text.strip() == '':
        return ''  # Return an empty string if the input is None or empty
    translator = Translator()
    translation = translator.translate(text, src='en', dest=ln)
    other_translation = translation.text
    return other_translation

In [42]:
lang=input("Enter language to be translated to: ").lower()

Enter language to be translated to: tamil


In [43]:
if lang not in language_codes.keys():
  print("Enter valid language")

In [44]:
langcode = language_codes[lang.lower()]
langcode

'ta'

In [45]:
english_text = data[:50]
other_text = translate_english_to_other(english_text,langcode)

print("English: ", english_text)
print("Translated: ", other_text)

English:  00:00:00,410 --> 00:00:01,258
Let's practice.
00:0
Translated:  00: 00: 00,410 -> 00: 00: 01,258
பயிற்சி செய்யலாம்.
00: 0


In [46]:
sentences=[]
tempsent=''
for i in data:
    if i!="\n":
        tempsent+=i
    else:
        sentences.append(tempsent)
        tempsent=''

In [47]:
f = open('subtitle.srt', 'w')

i=j=0
for sentence in sentences:
    i+=1
    if i%2==1:
        j+=1
        f.write(str(j))
        f.write("\n")
        f.write(sentence)
        f.write("\n")
    else:
        other_text = translate_english_to_other(sentence,langcode)
        f.write(other_text)
        f.write("\n\n")
f.close()

In [48]:
from google.colab import files

print('If you want to download enter 0 else enter 1 :- ')
m=int(input(''))
if m==0:
  transcript_file_path = '/content/subtitle.srt'
  try:
    files.download(transcript_file_path)
    print("Thank you! Downloaded successfully")
  except OSError as e:
      print(f"An error occurred while opening the file: {e}")
elif m==1:
  print('You do not want to download.Thank You...')
else:
  print('Sorry Wrong input.')

If you want to download enter 0 else enter 1 :- 
0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Thank you! Downloaded successfully
